Lisp has assisted a number of our most gifted fellowhumans in thinking previously impossible thoughts.

— Edsger Dijkstra

# Listas impróprias

Chamamos listas no formato `(x . y)` de listas impróprias, expliquei na segunda aula de lisp que listas no formato `(x y)` são equivalente a `x -> y -> '()`, já listas impróprias como `(x . y)` são equivalentes a `x -> y`.

#### Questão

O que o código `'(x y (z))` computa?

O que o código `'(x y . (z))` computa?


## Funções com número variável de argumentos

Listas impróprias nos permitem definir funções número variável de argumentos.

Veja:

In [3]:
(define (teste . args)
  (format #t "~A\n" args))

(teste)
(teste 1)
(teste 1 2)
(teste 1 2 3)

()
(1)
(1 2)
(1 2 3)
#t

Podemos também definir um número minimo de argumentos:

In [7]:
(define (teste arg1 arg2 . args)
  (format #t "~A ~A ~A\n" arg1 arg2 args))

(teste 1 2)
(teste 1 2 3)
(teste 1 2 3 4)

1 2 ()
1 2 (3)
1 2 (3 4)
#t

Perceba que o que vem após o ponto (`args`) está sempre dentro de uma lista, é assim que o **scheme** encapsula um número variável de argumentos, por baixo dos panos ele só roda um `(list a1 a2 a3 ... an)` tal que `ai` representa o i-ésimo argumento.

### Exemplo

Imagine que a função `+` recebe somente dois argumentos e queremos escrever uma versão que receba um número variável de argumentos, fariamos algo assim:

In [5]:
(define (soma . args)
  (cond [(null? args)       0]           ; 0 argumentos (soma)
        [(null? (cdr args)) (car args)]  ; 1 argumento (soma x)
        [else               (apply soma  ; Recursão
                                   (+ (car args) (cadr args)) 
                                   (cddr args))]))

(define (display\n . res)
    (for-each (lambda (x) 
                (format #t "~A\n" x)) 
              res))

(display\n (soma)
           (soma 1)
           (soma 1 -2)
           (soma -1 2 3))

0
1
-1
4


Sim, esse `apply` é o mesmo que implementamos na aula [Implementando o LISP em LISP](https://www.youtube.com/watch?v=yxhRQNT9FnM), ele recebe uma função `f` e um número variável de argumentos, sendo o último uma lista, e aplica recursivamente até que o último elemento da lista seja consumido.

Na aplicação acima ele é similar ao nosso `accumulate`.

### Questão
Complete a função `subtrai`, assumindo que a função `-` recebe 1 ou 2 argumentos.

Sendo `(- x)` equivalente a `-x` e `(- x y)` equivalente a `x - y`.

In [ ]:
(define (subtrai . args)
  ...)

(display\n (soma)         ; -> 0
           (soma 1)       ; -> -1
           (soma 1 -2)    ; -> 3
           (soma -1 2 3)) ; -> -6

# Abstração de padrões

Funções (procedimentos, métodos, etc...) são ótimas para abstrair padrões recorrentes em um contexto.

Existem padrões tão recorrentes em certas linguagens que ganham uma função na bilioteca padrão.

Por exemplo, ordenar um vetor/lista é tão comum que linguagens como `python`, `java`, `javascript` e `scheme` possuem a função `sort()`:

Python:
```python
arr = [2, 1, 4, 3]

arr.sort() # -> [1, 2, 3, 4]
```

Java:
```java
int arr[] = {2, 1, 4, 3};

Arrays.sort(arr) // -> [1, 2, 3, 4]
```

JavaScript:
```javascript
const arr = [2, 1, 4, 3]

arr.sort() // -> [1, 2, 3, 4]
```

Scheme:
```scheme
(define arr '(1 2 3 4))

(sort arr <) ; -> 4
```

## O poder de definir funções

Grande parte das vezes detectamos padrões recorrentes no nosso código e, com isso, escrever uma função para abstrair esses padrões.

Por exemplo, no código:

In [8]:
(define x 3)

(if (and (> x 0) (< x 5))
    (if (and (> x 1) (< x 4))
        (if (and (> x 2) (< x 3))
            x
            2)
        1)
    0)

2

In [9]:
(define y 2.5)

(if (and (> y 0) (< y 5))
    (if (and (> y 1) (< y 4))
        (if (and (> y 2) (< y 3))
            y
            2)
        1)
    0)

2.5

#### Questão
Quais são os 2 padrões recorrentes?

#### Questão
Defina duas funções que abstraem esses 2 padrões:

### O que fazer quando o padrão recorrente é uma estrutura da linguagem?

Diversas vezes encontramos padrões que não podem ser abstraidas em tempo de execução do código, são padrões presentes na estrutura do código.

Por exemplo, um padrão comum encontrado em muitas linguagens são loops `for`:

```C
for (int x = 0; x <= 10; x++) {
    printf("%d\n", x);
}
```

Normalmente a "chamada" de um loop `for` recebe: um ou mais simbolos inicializados (`x` é inicializado com `0`), uma ou mais condicionais (`x <= 10`), um ou mais passos (`x++`) e um corpo (`printf("%d\n", x);`). O corpo será repetido enquanto as condicionais forem verdadeiras, a cada iteração os passos são aplicados.

Esse código erá printar o valor de `x` (inicialmente 0), enquanto `x` for menor ou igual a `10` e, em cada iteração, irá somar um a `x`. ou seja, irá printar todos os inteiros de 0 a 10.

### Podemos definir `for` como uma função?

Perceba que o loop `for` recebe trechos de código, não váriaveis, ou seja, ele manipula a estrutura da linguagem, não valores definidos em tempo de execução.

Entretanto, em **LISPs** é sim possível definir uma função que se comporta como um `for`.

Isso ocorre pois em **LISPs** código é dado e dado é código, algo incomum para outras linguagens...

Por exemplo:

In [1]:
(car '(car '(x y)))

car

Acima nós passamos um trecho de código (car '(x y)) como valor para uma funçõa `car`, utilizando `'` (ou `quote`).

Sendo assim, poderiamos escrever uma função `for` que recebe uma lista de pares `(var init)`, uma lista de condicionais, uma lista de passos e um corpo.

Uma chamada da função `for` seria algo como:

```scheme
(for '((x 1)) '((<= x 10)) '((1+ x)) '(format #t "~A\n" x))
```

Perceba que a chamada acima é bem incomum, apesar de conseguirmos utilizar `quote` para criar listas que representam código, isso não é muito comum, escrever a chamada acima não mantém uma homogeneidade muito grande com o resto do código, espera-se que chamadas de função recebam valores, não códigos.

Se pudessemos reescrever o loop como:

```scheme
(for ((x 1)) ((<= x 10)) ((1+ x)) (format #t "~A\n" x))
```

Seria muito mais parecido com o que encontramos em scheme...

#### Questão
Por que não seria possível o código supracitado se `for` fosse uma função?

### O que não é dito...
A verdade é que estruturas como loops `for` e `while` são analisados em tempo de compilação, expandidos para códigos mais complicados e incomuns.

**ACTUALLY ☝️🤓** em **C** loops `for` e `while` são transformados diretamente para código de máquina, não passam pelo preprocessador...

Seria tão bom se pudessemos rodar funções em tempo de compilação...

## MACROS!!!!

Em resumo, macros são funções executadas em tempo de compilação

Por exemplo, o código:

In [5]:
(define x "sou uma string")

(cond [(number? x) 'number]
      [(list? x)   'list]
      [(char? x)   'char]
      [(string? x) 'string]
      [else        'something-else])

string

Expande, em tempo de compilação para:

In [6]:
(if (number? x)
    'number
    (if (list? x)
        'list
        (if (char? x) 
            'char
            (if (string? x)
                'string
                'something-else))))

string

### Definindo macros

Em scheme temos algumas formas de definir macros, nessa aula vamos cobrir 2 que na verdade são a mesma: `defmacro` e `define-macro`.

### Um pouco de contexto

Em Common Lisp é definimos funções com a sintaxe `(defun nome (args) corpo)`, diferentemente da sintaxe do scheme, onde nome a args compõe a mesma lista `(define (nome args) corpo)`.

O mesmo é equivalente para `defmacro` e `define-macro`, em common lisp fazemos `(defmacro nome (args) body)` enquanto em scheme fazemos
`(define-macro (nome args) body)`.

Guile nos disponibiliza as duas formas, sendo que o primeiro é só uma chamadda para o segunda.

### Retornando

Definir um macro é muito parecido com a definição de uma função, vamos definir um macro que soma 1 a um número:

In [1]:
(define-macro (macro-1+ x)
  (1+ x))

(macro-1+ 2)

3

Simples, certo?

Mas nem tudo são rosas... 

#### Questão
O que acontece se passarmos a chamada: `(car '(1 2 3))` para `macro-1+ x`?

Macros são diferentes de funções pois não computam seus argumentos, nesse caso, a chamada `(car '(1 2 3))` é examente isso para um macro, veja:

In [4]:
(define-macro (macro-display x)
  (format #t "MACRO: ~A\n" x))

(define (func-display x)
  (format #t "FUNÇÃO: ~A\n" x))

(func-display (car '(1 2 3)))

(macro-display (car '(1 2 3)))

MACRO: (car (quote (1 2 3)))
FUNÇÃO: 1
#t

Percebe que a chamada de `macro-display` retorna `(car (quote (1 2 3)))` e de `func-display` retorna `1`.

Mas há algo errado, a chamado de `macro-display` ocorre após `func-display` e mesmo assim seu print aparece antes...

Isso ocorre pois macros são computados em tempo de **compilação** e função são computadas em tempo de **execução**.

### Muito legal mas... como isso é útil?

É comum macros devolverem listas (código) não só aplicarem uma função em tempo de compilação.

Isso ocorre pois, como explicado anteriormente, macros são computados em tempo de **compilação**, isso significa que, seu resultado será computado em tempo de **execução**.

Como assim?


In [9]:
(define-macro (macro-soma->lista x y)
  (list 'list (list '+ x y)))

(macro-soma->lista 1 2)

(3)

Vejamos o equivalente escrito como função:

In [10]:
(define (func-soma->lista x y)
  (list 'list (list '+ x y)))

(func-soma->lista 1 2)

(list (+ 1 2))

O macro, em tempo de compilação, retorna a lista `(list (+ 1 2))`, ou melhor, a chamada `(macro-soma->lista 1 2)` é substituida pela lista `(list (+ 1 2))` que, por sua vez, em tempo de execução é computada:

In [11]:
(list (+ 1 2))

(3)

#### Pipeline

A pipeline é algo como:

Código -> Parser/Leitor -> Expansão de Reader Macros -> Expansão de Macros -> Compilador -> Código de Máquina (ou bytecode) -> Execução 

Onde Reader Macros são macros que são expandidos em tempo de leitura/parser, como o `'` que é expandido para o macro `quote`:

In [2]:
''x

(quote x)

Não falaremos sobre a criação de reader macros nessa disciplina, mas saibam que é possível em **Common Lisp** com certa facilidade e em **scheme** com algumas restrições.

#### Questão
Escreva um macro `infix->prefix` que recebe um código em formato infixado `(x operador y)`, inapropriado para o **scheme**, e transforma em código prefixado `(operador x y)`, que pode ser computado pelo **scheme**.

Exemplo: `(1 + 2) -> (+ 1 2) -> 3`

In [ ]:
(define-macro (infix->prefix infix)
  (let ([op (?   infix)]
        [v1 (??  infix)]
        [v2 (??? infix)])
    (list ???? ????? ??????)))